# import libs

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from FCNN import TwoInputFCNN as FCNN
from sklearn.preprocessing import MinMaxScaler


In [ ]:
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

# import data

In [ ]:
# paths pre-setting
label_root = f'/home/{os.getlogin()}/ERIE/silicone/output'
model_root = f'/home/{os.getlogin()}/ERIE/silicone/Track-Shuyuan-2023-08-13/videos'
model_name = 'DLC_resnet50_TrackAug13shuffle1_50000'


In [ ]:
# pre setting training sets
training_sets = ['C_M1_T1_4', 'C_M1_T1_7', 'C_M1_T1_3', 'C_M1_T1_6',
                 'R2_M1_T1_6', 'R2_M1_T1_7', 'R2_M1_T1_3', 'R2_M1_T1_5',
                 'L2_M1_T1_3', 'L2_M1_T1_4', 'L2_M1_T1_5', 'L2_M1_T1_7',
                 'Z2_M1_T1_1', 'M3_Z2_NF', 'M5_Z2_NF', 'M12_Z2_NF']

# initial arrays
X_train_L = np.zeros((0, 16))
X_train_R = np.zeros((0, 16))
y_train = np.zeros((0, 3))

for set in tqdm(training_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    coordinates_L = coordinates_L.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    coordinates_R = coordinates_R.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    labels = labels[:frames, 7:10]

    X_train_L = np.vstack(
        (X_train_L, MinMaxScaler().fit_transform(coordinates_L)))
    X_train_R = np.vstack(
        (X_train_R, MinMaxScaler().fit_transform(coordinates_R)))
    y_train = np.vstack((y_train, MinMaxScaler().fit_transform(labels)))


In [ ]:
X_train_L.shape, X_train_R.shape, y_train.shape

In [ ]:
# pre setting validation sets
val_sets = ['C_M1_T1_1', 'C_M1_T1_5', 'R2_M1_T1_2', 'R2_M1_T1_4',
            'L2_M1_T1_6', 'L2_M1_T1_1', 'Z2_M1_T1_2', 'M7_Z2_NF']

# initial arrays
X_val_L = np.zeros((0, 16))
X_val_R = np.zeros((0, 16))
y_val = np.zeros((0, 3))

for set in tqdm(val_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    coordinates_L = coordinates_L.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    coordinates_R = coordinates_R.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    labels = labels[:frames, 7:10]

    X_val_L = np.vstack(
        (X_val_L, MinMaxScaler().fit_transform(coordinates_L)))
    X_val_R = np.vstack(
        (X_val_R, MinMaxScaler().fit_transform(coordinates_R)))
    y_val = np.vstack((y_val, MinMaxScaler().fit_transform(labels)))


In [ ]:
X_val_L.shape, X_val_R.shape, y_val.shape

# Trainning and validation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = FCNN(input_dim=X_train_L.shape[1], hidden_dim=16, output_dim=y_train.shape[1],
           device=device, weights=[0.5, 1.0, 0.5], num_hidden_layers=4, batch_size=32,
           l2_reg=0.0001, lr=0.0001, random_seed=42, message='silicone R')
net.train(X_train_R, X_train_L, y_train, X_val_R, X_val_L,
          y_val, epochs=200, use_tqdm=True, save_loss=True)


In [ ]:
indices = np.random.permutation(len(X_train_R))

X_train_R_shuffled = X_train_R[indices]
X_train_L_shuffled = X_train_L[indices]
y_train_shuffled = y_train[indices]

In [ ]:
for percentage in [0.25, 0.5, 0.75, 1.0]:
    end_idx = int(len(X_train_R_shuffled) * percentage)
    X_train_R_slice = X_train_R_shuffled[:end_idx:45]
    X_train_L_slice = X_train_L_shuffled[:end_idx:45]
    y_train_slice = y_train_shuffled[:end_idx:45]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = FCNN(input_dim=X_train_L.shape[1], hidden_dim=16, output_dim=y_train.shape[1],
               device=device, weights=[0.5, 1.0, 0.5], num_hidden_layers=4, batch_size=32,
               l2_reg=0.0001, lr=0.0001, random_seed=42, 
               message=f"Data usage: {percentage*100:.0f}%")
    net.train(X_train_R_slice, X_train_L_slice, y_train_slice, X_val_R, X_val_L,
              y_val, epochs=200, use_tqdm=True, save_loss=True)


# Retrieve model

In [ ]:
net_name = input()

In [ ]:
import json

with open(f'results/losses_{net_name}.json', 'r') as f:
    params = json.load(f)

input_dim = params["input_dim"]
hidden_dim = params["hidden_dim"]
output_dim = params["output_dim"]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = params["weights"]
num_hidden_layers = params["num_hidden_layers"]
lr = params["lr"]
l2_reg = params["l2_reg"]
batch_size = params["batch_size"]
random_seed = params["random_seed"]

net = FCNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim,
           device=device, weights=weights, num_hidden_layers=num_hidden_layers,
           batch_size=batch_size, lr=lr, l2_reg=l2_reg, random_seed=random_seed)


In [ ]:
net.load_state_dict(torch.load(
    f'models/best_model_{net_name}.pth', map_location=device))


# Test

In [ ]:
# pre setting test sets
test_sets = ['C_M1_T1_8', 'M1_NF', 'M2_NF', 'R1_M1_T1_1',
             'R1_M1_T1_2', 'R3_M1_T1_1', 'R3_M1_T1_2', 'L1_M1_T1_1',
             'L1_M1_T1_2', 'L3_M1_T1_1', 'L3_M1_T1_2', 'Z1_M1_T1_1',
             'Z3_M1_T1_1', 'R2_M1_T1_8', 'M3_R2_NF', 'M5_R2_NF',
             'M5_L2_NF', 'M3_L2_NF', 'M7_L2_NF', 'Z2_M1_T1_5',
             'M9_Z2_NF', 'M8_Z2_NF']

# initial arrays
rmse = np.zeros((3, len(test_sets)))

for i, set in enumerate(test_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    coordinates_L = coordinates_L.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    coordinates_R = coordinates_R.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    labels = labels[:frames, 7:10]

    X_test_L = MinMaxScaler().fit_transform(coordinates_L)
    X_test_R = MinMaxScaler().fit_transform(coordinates_R)
    y_test = MinMaxScaler().fit_transform(labels)
    y_pred = net.predict(X_test_R, X_test_L)

    # save prediction
    # np.savetxt(f"labels/{set}_fcnn.txt", y_pred, delimiter=",")

    from sklearn.metrics import mean_squared_error
    for axis in range(3):
        rmse[axis, i] = mean_squared_error(y_test[:, axis], y_pred[:, axis], squared=False)

    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    fig.suptitle(f"Position - Time plot of imageset {set}")
    titles = ['X', 'Y', 'X']
    fps = 30  # Frames per second
    time_values = [i/fps for i in range(y_test.shape[0])]
    for i in range(3):
        ax[i].plot(time_values, y_test[:, i], color='black', label='actual')
        ax[i].plot(time_values, y_pred[:, i], color='red', label='predict')
        if i == 1:
            ax[i].legend()
        ax[i].set_ylabel(f'norm position {titles[i]}')
        ax[i].set_xlim([0, np.max(time_values)])
        ax[i].set_ylim([-0.1, 1.1])

    # Set the x-label only for the bottom subplot
    ax[-1].set_xlabel('Time (s)')

    plt.tight_layout()
    # plt.savefig(os.path.join(os.getcwd(), f'plots/{set}.pdf'), format='pdf')
    import pickle
    with open(os.path.join(os.getcwd(), f'plots/{set}_fcnn.pickle'), 'wb') as f:
        pickle.dump(fig, f)
    plt.close()


In [ ]:
# Calculate mean and standard deviation of RMSE for all dimensions
mean_rmse = np.mean(rmse, axis=1)
std_rmse = np.std(rmse, axis=1)

# Print mean and std for all dimensions
print("Average RMSE (mean ± std):")
print(f"{np.mean(mean_rmse):.3f} ± {np.mean(std_rmse):.3f}")

# Calculate mean and standard deviation of RMSE for each dimension and model
dimensions = ['x', 'y', 'z']
print("\nRMSE (mean ± std) for each dimension:")
for i, dim in enumerate(dimensions):
    print(f"{dim}: {mean_rmse[i]:.3f} ± {std_rmse[i]:.3f}")
